Global Variables

In [168]:
file_path = "ITIL Books/ITIL 3/ITIL3 Service Operation chapter 4.pdf"
# extracted_text_file_path = "ITIL Books/ITIL 3/Continual service improvement chapter from notebook.txt"
# extracted_text_file_path = "ITIL Books/ITIL 3/Service operation chapter 4/Service operation chapter 4 - 4.txt"
# extracted_text_file_path = "ITIL Books\ITIL 3\Service operation chapter 4\Automated concepts extracted\\4.2\Service operation chapter 4 - 4.2 to 4.2.4 .txt"
output_file_path = "output/ITIL3 Continual Service Improvement.txt"

### Chapter 4 start ###
# extracted_text_file_path = "ITIL Books/ITIL 3/Service operation chapter 4/Service operation chapter 4 - 4.txt"
# manual_concepts_file_path = "ITIL Books/ITIL 3/Service operation chapter 4/Automated concepts extracted/4/Manual concepts extracted 4.txt"

### Chapter 4 - 4.1 to 4.1.4 ###
extracted_text_file_path = "ITIL Books\ITIL 3\Service operation chapter 4\Automated concepts extracted\\4.1\Service operation chapter 4 - 4.1 to 4.1.4.txt"
manual_concepts_file_path = "ITIL Books/ITIL 3/Service operation chapter 4/Automated concepts extracted/4.1/Manual concepts extracted 4.1.txt"

### Chapter 4 - 4.2 to 4.2.4 ###
# extracted_text_file_path = "ITIL Books\ITIL 3\Service operation chapter 4\Automated concepts extracted\\4.2\Service operation chapter 4 - 4.2 to 4.2.4.txt"
# manual_concepts_file_path = "ITIL Books/ITIL 3/Service operation chapter 4/Automated concepts extracted/4.2/Manual concepts extracted 4.2.txt"

Imports

In [169]:
import pdfminer
import nltk
import re

Global Functions

In [170]:
with open(extracted_text_file_path, 'r') as file:
    extracted_text = file.read()
    
# extracted_text = "Particle dynamics involves the study of physics and chemistry"

# tokens = nltk.word_tokenize(extracted_text)
# print(tokens)

# ### Part of speech tagging ###
# part_of_speech_array = nltk.pos_tag(tokens)
# print(part_of_speech_array)


Text sanitization and word tokenizing

In [171]:
### Grab sections from text ###
# print(re.findall("^\d(\.|\d)*(\s|\w)*$", extracted_text))
# title_pattern = re.compile(r"^\d(\.|\d)*(\s|\w)*$", re.MULTILINE)
title_pattern = re.compile(r"^\d+.*$", re.MULTILINE)

sections = title_pattern.findall(extracted_text)
for counter, section in enumerate(sections):
    if not (section.find("%") == -1 and section.find(")") == -1):
        sections.remove(section)

### Sanitise extracted text ###
extracted_text_sanitised = extracted_text
extracted_text_sanitised = extracted_text.replace("¦", "")
extracted_text_sanitised = extracted_text_sanitised.replace("–", "")
        
### Tokenise extracted text ###
tokens = nltk.word_tokenize(extracted_text_sanitised)
# print(tokens)


Part to speech tagging

In [172]:
### Part of speech tagging ###
part_of_speech_array = nltk.pos_tag(tokens)
# print(part_of_speech_array)

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

part_of_speech_array_lemmatized = []

for part_of_speech in part_of_speech_array:
    part_of_speech_array_lemmatized.append(
        (lemmatizer.lemmatize(part_of_speech[0]), part_of_speech[1])
    ) 
    
# print(part_of_speech_array_lemmatized)


Term extraction

In [173]:
### Term Extraction (NNP next to each other) ###
def extract_terms(part_of_speech_array_array, tags_to_use):
    terms_array = []
    term_phrase = []
    start_new_term = True
    for index, part in enumerate(part_of_speech_array_array):
        if(part[1] in tags_to_use):
            term_phrase.append(part[0])
            start_new_term = False if part_of_speech_array_array[index + 1][1] in tags_to_use else True

            if start_new_term == True:
                terms_array.append(" ".join(term_phrase))
                term_phrase = []
    return terms_array

### Term Extraction (NNP next to each other) ###
def extract_terms_with_adj(part_of_speech_array_array, tags_to_use):
    terms_array = []
    term_phrase = []
    start_new_term = True
    for index, part in enumerate(part_of_speech_array_array):
        if(part[1] in tags_to_use):
            term_phrase.append(part[0])
            start_new_term = False if part_of_speech_array_array[index + 1][1] in tags_to_use else True

            if start_new_term == True:
                terms_array.append(" ".join(term_phrase))
                term_phrase = []
    return terms_array

major_named_concepts = extract_terms(part_of_speech_array, {"NNP", "NNPS"})
other_concepts = extract_terms(part_of_speech_array, {"NN", "NNS"})
all_noun_phrases = extract_terms(part_of_speech_array, {"NNP", "NNPS", "NN", "NNS"})
all_noun_phrases_with_adj = extract_terms_with_adj(part_of_speech_array, {"NNP", "NNPS", "NN", "NNS", "JJ"})

print(f"major named concepts: \n{major_named_concepts}")
print(f"\nother concepts: \n{other_concepts}")
print(f"\nall noun phrases: \n{all_noun_phrases}")
print(f"\nall noun phrases with adj: \n{all_noun_phrases_with_adj}")


major named concepts: 
['INCIDENT MANAGEMENT', 'ITIL', 'IT', 'IT', 'Management', 'Service Desk', 'Purpose/goal/objective', 'Incident Management', '‘ Normal', '’', 'SLA', 'Scope Incident Management', 'Service Desk', 'Event Management', 'Incident Management', 'Service Desk', 'Service Desk', 'Service', '’', 'SLA', 'Service', 'Request Fulfilment', 'Value', 'Incident Management', 'IT', 'Incident Management', 'Service Desk', 'IT', 'Incident Management', 'Service Operation', 'Incident Management', 'Service Management', 'Management']

other concepts: 
['terminology', 'incident ’', 'interruption', 'service', 'reduction', 'quality', 'service', 'Failure', 'configuration item', 'service', 'incident', 'example failure', 'disk', 'mirror set', 'process', 'incidents', 'failures', 'questions', 'queries', 'users', 'telephone call', 'staff', 'event monitoring tools', 'goal', 'process', 'service operation', 'impact', 'business operations', 'levels', 'service quality', 'availability', 'service operation', 

Major/common concept extraction

In [174]:
# concept_relationships = extract_terms(part_of_speech_array, {"VP"})
# print(concept_relationships)
# print(all_noun_phrases)

### Perform frequency analysis ###
### Concept Extraction Frequency analysis ###
major_named_concept_frequency_distribution = nltk.FreqDist(major_named_concepts)
other_concept_frequency_distribution = nltk.FreqDist(other_concepts)
all_noun_phrases_frequency_distribution = nltk.FreqDist(all_noun_phrases)

print(f"major named concepts: \n{major_named_concept_frequency_distribution.most_common(50)}")
print(f"\nother concepts: \n{other_concept_frequency_distribution.most_common(50)}")
print(f"\nall noun phrases: \n{all_noun_phrases_frequency_distribution.most_common(50)}")

major named concepts: 
[('Incident Management', 6), ('Service Desk', 5), ('IT', 4), ('Management', 2), ('’', 2), ('SLA', 2), ('Service', 2), ('INCIDENT MANAGEMENT', 1), ('ITIL', 1), ('Purpose/goal/objective', 1), ('‘ Normal', 1), ('Scope Incident Management', 1), ('Event Management', 1), ('Request Fulfilment', 1), ('Value', 1), ('Service Operation', 1), ('Service Management', 1)]

other concepts: 
[('service', 8), ('business', 6), ('incidents', 5), ('incident', 3), ('process', 3), ('staff', 3), ('service operation', 3), ('events', 3), ('ability', 3), ('users', 2), ('availability', 2), ('section', 2), ('requests', 2), ('value', 2), ('areas', 2), ('terminology', 1), ('incident ’', 1), ('interruption', 1), ('reduction', 1), ('quality', 1), ('Failure', 1), ('configuration item', 1), ('example failure', 1), ('disk', 1), ('mirror set', 1), ('failures', 1), ('questions', 1), ('queries', 1), ('telephone call', 1), ('event monitoring tools', 1), ('goal', 1), ('impact', 1), ('business operations

Concept relationship extraction

In [175]:
def get_sentence_at_index(part_of_speech_array, index):
    sentence_starting_index = 0
    sentence_end_index = len(part_of_speech_array)
    
    ### Get sentence start index ###
    for i in range(0, index):
#         print(part_of_speech_array[index])
        if part_of_speech_array[index - i][1] == ".":
            sentence_starting_index = index - i
            break
    
    ### Get sentence end index ###
    for i in range(0, len(part_of_speech_array)):
        if part_of_speech_array[index + i][1] == ".":
            sentence_end_index = index + i
            break
            
    return (sentence_starting_index, sentence_end_index + 1)

### Term Extraction (NNP next to each other) ###
def extract_terms(part_of_speech_array_array, tags_to_use):
    part_of_speech_array_with_terms = []
    
    terms_array = []
    term_phrase = []
    start_new_term = True
    for index, part in enumerate(part_of_speech_array_array):
        if(part[1] in tags_to_use):
            term_phrase.append(part[0])
            start_new_term = False if part_of_speech_array_array[index + 1][1] in tags_to_use else True

            if start_new_term == True:
                if len(term_phrase) > 1:
#                     part_of_speech_array_with_terms.append((" ".join(term_phrase), f"NPhrase-{part[1]}"))
                    part_of_speech_array_with_terms.append((" ".join(term_phrase), "NPhrase"))
                else:
                    part_of_speech_array_with_terms.append((" ".join(term_phrase), part[1]))
                term_phrase = []
        else:
            part_of_speech_array_with_terms.append((part[0], part[1]))
    return part_of_speech_array_with_terms

part_of_speech_array_with_terms = extract_terms(part_of_speech_array, {"NNP", "NNPS", "NN", "NNS"})
# print(part_of_speech_array_with_terms)

sentences = []
temp_sentence = []
for word in part_of_speech_array_with_terms:
    if word[1] is ".":
        temp_sentence.append(word)
        sentences.append(temp_sentence)
        temp_sentence = []
    else:
        temp_sentence.append(word)
        
        
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# for index, sentence in enumerate(sentences):
#     sentence_with_no_stop_words = [word_pos for word_pos in sentence if not word_pos[0] in stop_words]
#     
#     if index > 3:
#         break
#         
#     print(sentence)
#     print(sentence_with_no_stop_words)
#     print("-" * 100)


In [176]:
# from part_of_speech_regex import PartOfSpeechRegex

class PartOfSpeechRegex:
    def parseAndReturnPatterns(self, pattern_string, sentence):
        print(pattern_string)

pattern_string = "*<JJ><NNP><VBZ><NP>"

print(sentences[3])

part_of_speech_regex = PartOfSpeechRegex()
# part_of_speech_regex.parseAndReturnPatterns()

part_of_speech_regex.parseAndReturnPatterns(pattern_string, sentences[3])


[('4.2.1', 'CD'), ('Purpose/goal/objective', 'NNPS'), ('The', 'DT'), ('primary', 'JJ'), ('goal', 'NN'), ('of', 'IN'), ('the', 'DT'), ('Incident Management process', 'NPhrase'), ('is', 'VBZ'), ('to', 'TO'), ('restore', 'VB'), ('normal', 'JJ'), ('service operation', 'NPhrase'), ('as', 'RB'), ('quickly', 'RB'), ('as', 'IN'), ('possible', 'JJ'), ('and', 'CC'), ('minimize', 'VB'), ('the', 'DT'), ('adverse', 'JJ'), ('impact', 'NN'), ('on', 'IN'), ('business operations', 'NPhrase'), (',', ','), ('thus', 'RB'), ('ensuring', 'VBG'), ('that', 'IN'), ('the', 'DT'), ('best', 'JJS'), ('possible', 'JJ'), ('levels', 'NNS'), ('of', 'IN'), ('service quality', 'NPhrase'), ('and', 'CC'), ('availability', 'NN'), ('are', 'VBP'), ('maintained', 'VBN'), ('.', '.')]
*<JJ><NNP><VBZ><NP>


In [177]:
most_common_major_concepts = major_named_concept_frequency_distribution.most_common(50)
# print(most_common_major_concepts)
# print(tokens.index(most_common_major_concepts[0][0]))
# print(part_of_speech_array[206])
## Get indices of all common concepts
indices = [i for i, x in enumerate(part_of_speech_array) if x[0] == most_common_major_concepts[0][0]]
# print(indices)
# print(most_common_major_concepts[1][0])

def get_sentence_at_index(part_of_speech_array, index):
    sentence_starting_index = 0
    sentence_end_index = len(part_of_speech_array)
    
    ### Get sentence start index ###
    for i in range(0, index):
        if part_of_speech_array[index - i][1] == ".":
            sentence_starting_index = index - i
            break
    
    ### Get sentence end index ###
    for i in range(0, index):
        if part_of_speech_array[index + i][1] == ".":
            sentence_end_index = index + i
            break
            
    return (sentence_starting_index + 1, sentence_end_index + 1)

def does_list_contain_verb_pos(part_of_speech_array):
    for word_pos in part_of_speech_array:
        if word_pos[1] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
            return True
    return False
    
# sentence_index = get_sentence_at_index(part_of_speech_array, indices[0])
# print(sentence_index)

concept_relations = []

i = 0
for index in indices:
#     if i < 3:
#         i = i + 1
#         continue
    sentence_pos_containing_concept = part_of_speech_array[
        get_sentence_at_index(part_of_speech_array, index)[0]:
        get_sentence_at_index(part_of_speech_array, index)[1]
    ]
    print(sentence_pos_containing_concept)
        
    last_concept = ()
    last_concept_index = -1
    # For word part_of_speech in sentence_part_of_speech_containing_concept
    for index, word_pos in enumerate(sentence_pos_containing_concept):
#         print(f"{word_pos[0]}: {word_pos[1]}")
#         print(word_pos[0] in all_noun_phrases)
        
#         if (word_pos[0] in all_noun_phrases):
        if (word_pos[0] in major_named_concepts):
            if last_concept_index != -1:# and does_list_contain_verb_pos(sentence_pos_containing_concept[last_concept_index + 1:index]):
                concept_relations.append(f"{last_concept}::{sentence_pos_containing_concept[last_concept_index + 1:index]}::{word_pos}")
            
            last_concept = word_pos
            last_concept_index = index
        
    # print(related_concepts)
    # print()
    
#     print('-----')
    i = i + 1
#     if i == 4:
#         break

# for concept_relation in concept_relations:
#     print(concept_relation)


Metrics for term extraction chapter 4 first section

In [178]:
# automatic_concepts_file_path = "ITIL Books/ITIL 3/Service operation chapter 4/Automated concepts extracted/4/Automated concepts extracted 4.txt"
# manual_concepts_file_path = "ITIL Books/ITIL 3/Service operation chapter 4/Automated concepts extracted/4/Manual concepts extracted 4.txt"

# automatic_concepts_file_path = "ITIL Books/ITIL 3/Service operation chapter 4/Automated concepts extracted/4.2/Automated concepts extracted 4.2.txt"
# manual_concepts_file_path = "ITIL Books/ITIL 3/Service operation chapter 4/Automated concepts extracted/4.2/Manual concepts extracted 4.2.txt"
# 
# with open(automatic_concepts_file_path, 'r') as file:
#     automatic_concepts = file.read()

with open(manual_concepts_file_path, 'r') as file:
    manual_concepts = file.read()

manual_concepts_list = manual_concepts.split('\n')
manual_concepts_list = [x.lower() for x in manual_concepts_list]

# print("Manual concepts")
# print(list(dict.fromkeys(manual_concepts_list)))
# print()

# automatic_concepts_list = ['Service Operation', 'processes', 'paragraph', 'detail', 'chapter', 'reference', 'structure', 'processes', 'detail', 'chapter', 'Please note', 'roles', 'process', 'tools', 'process', 'Chapters', 'Management', 'process', 'monitors', 'events', 'IT infrastructure', 'operation', 'exception conditions', 'Incident Management', 'service', 'users', 'order', 'business impact', 'Problem Management', 'root-cause analysis', 'cause', 'events', 'incidents', 'activities', 'problems/incidents', 'Known Error subprocess', 'quicker diagnosis', 'resolution', 'incidents', 'NOTE', 'distinction', 'incidents', 'problems', 'Incident', 'Problem Records', 'danger', 'Incidents', 'support cycle', 'actions', 'recurrence', 'incidents', 'Incidents', 'root cause analysis', 'visibility', 'user ’ s service', 'SLA targets', 'service', 'users', 'expectations', 'results', 'number', 'incidents', '‘ purge ’', 'visibility', 'issues', 'Request Fulfilment', 'management', 'customer', 'user requests', 'incident', 'service delay', 'disruption', 'organizations', 'requests', 'category ’', 'incidents', 'information', 'Incident Management system', 'others', 'volumes', 'business priority', 'requests', 'provision', 'Request Fulfilment', 'Request Fulfilment process', 'practice', 'Request Fulfilment process', 'customer', 'user requests', 'types', 'requests', 'facilities', 'moves', 'supplies', 'IT services', 'requests', 'SLA measures', 'records', 'process flow', 'practice', 'organizations', 'Access Management', 'process', 'users', 'right', 'service', 'access', 'users', 'users', 'ability', 'access services', 'stages', 'resources', 'HR', 'lifecycle', 'Access Management', 'Identity', 'Rights Management', 'organizations', 'addition', 'processes', 'Service Operation', 'phases', 'Service Management Lifecycle', 'aspects', 'processes', 'part', 'chapter', 'include', 'Change Management', 'process', 'Configuration Management', 'Release Management', 'topics', 'Service Transition publication', 'Capacity', 'Availability Management', 'aspects', 'publication', 'detail', 'Service Design publication', 'Financial Management', 'Service Strategy publication', 'Knowledge Management', 'Service Transition publication', 'IT Service Continuity', 'Service Design publication', 'Service Reporting', 'Measurement', 'Continual Service Improvement publication']
automatic_concepts_list = all_noun_phrases
automatic_concepts_list = [x.lower() for x in automatic_concepts_list]

# print("all noun phrases")
# print(list(dict.fromkeys(automatic_concepts_list)))

count = 0
for concept in manual_concepts_list:
    if concept in automatic_concepts_list:
        count = count + 1

number_of_fully_correct_manual_concepts = count

number_of_manual_concepts = len(manual_concepts_list)

count = 0
for concept in automatic_concepts_list:
    if concept in manual_concepts_list:
        count = count + 1
    
number_of_fully_correct_automatic_concepts = count

number_of_automatic_concepts = len(automatic_concepts_list)

print(f"number_of_manual_concepts: {number_of_manual_concepts}")
print(f"number_of_automatic_concepts: {number_of_automatic_concepts}")
print(f"number_of_fully_correct_manual_concepts: {number_of_fully_correct_manual_concepts}")
print(f"number_of_fully_correct_automatic_concepts: {number_of_fully_correct_automatic_concepts}")

# Lists to words for partial matches
automatic_concepts_list_single_words = [x.split() for x in automatic_concepts_list]
# print(automatic_concepts_list_single_words)

manual_concepts_list_single_words = [x.split() for x in manual_concepts_list]
# print(manual_concepts_list_single_words)

count = 0
for concept in manual_concepts_list_single_words:
    for word in concept:
        if word in ' '.join(automatic_concepts_list).split():
            count = count + 1
            break
        
number_of_full_and_partial_correct_manual_concepts = count
print(f"number_of_full_and_partial_correct_manual_concepts: {number_of_full_and_partial_correct_manual_concepts}")

count = 0
for concept in automatic_concepts_list_single_words:
    for word in concept:
        if word in ' '.join(manual_concepts_list).split():
            count = count + 1
            break
            
number_of_full_and_partial_correct_automatic_concepts = count
print(f"number_of_full_and_partial_correct_automatic_concepts: {number_of_full_and_partial_correct_automatic_concepts}")

number_of_manual_concepts: 104
number_of_automatic_concepts: 129
number_of_fully_correct_manual_concepts: 89
number_of_fully_correct_automatic_concepts: 91
number_of_full_and_partial_correct_manual_concepts: 101
number_of_full_and_partial_correct_automatic_concepts: 106
